In [1]:
import os
from dotenv import load_dotenv

from llama_index.llms.databricks import Databricks
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

from pinecone import Pinecone
from llama_index.core import Settings, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore


# Load environment variables
load_dotenv()

# API keys setup
DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
COHERE_API_KEY = os.environ.get('COHERE_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

# Setup embedding model
Settings.embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
)

# Setup Pinecone vector store
pc = Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pc.Index("sidindex")
index = PineconeVectorStore(pinecone_index=pinecone_index, text_key="text")


c:\Users\NailFerroukhi\miniconda3\envs\sid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\NailFerroukhi\miniconda3\envs\sid\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
from llama_index.core.query_pipeline import InputComponent, QueryPipeline, ArgPackComponent
from llama_index.core.prompts import PromptTemplate
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.memory import ChatMemoryBuffer
from scripts.pipeline import ChatPipeline, ResponseWithChatHistory



# First, we create an input component to capture the user query
input_component = InputComponent()

# Next, we use the LLM to rewrite a user query
rewrite = (
    "Please write a query to a semantic search engine using the current conversation.\n"
    "\n"
    "\n"
    "{chat_history_str}"
    "\n"
    "\n"
    "Latest message: {query_str}\n"
    'Query:"""\n'
)
rewrite_template = PromptTemplate(rewrite)

llm = Databricks(
    model="databricks-meta-llama-3-1-70b-instruct",
    api_key=DATABRICKS_TOKEN,
    api_base="https://adb-7215147325717155.15.azuredatabricks.net/serving-endpoints",
)

# using that, we will retrieve...
retriever = VectorStoreIndex.from_vector_store(index).as_retriever(similarity_top_k=15)

# we will retrieve two times, so we need to pack the retrieved nodes into a single list
argpack_component = ArgPackComponent()

# then postprocess/rerank with Cohere reranker
reranker = CohereRerank(api_key=COHERE_API_KEY, top_n=10)


response_component = ResponseWithChatHistory(
    llm=llm,
    system_prompt=(
        "You are a Q&A system. You will be provided with the previous chat history, "
        "as well as possibly relevant context, to assist in answering a user message."
    ),
)

p = QueryPipeline(verbose=False)
p.add_modules(
            {
                "input": input_component,
                "rewrite_template": rewrite_template,
                "llm": llm,
                "retriever": retriever,
                "reranker": reranker,
                "response_component": response_component,
            }
        )
p.add_link("input", "rewrite_template", src_key="query_str", dest_key="query_str")
p.add_link("input", "rewrite_template", src_key="chat_history_str", dest_key="chat_history_str")
        
# Linking rewrite_template to LLM, and LLM to retriever
p.add_link("rewrite_template", "llm")
p.add_link("llm", "retriever")

# Linking retriever to reranker, and LLM to reranker
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")

# Linking reranker to response_component
p.add_link("reranker", "response_component", dest_key="nodes")

# Linking input to response_component
p.add_link("input", "response_component", src_key="query_str", dest_key="query_str")
p.add_link("input", "response_component", src_key="chat_history", dest_key="chat_history")

pipeline_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

In [12]:
pipeline = QueryPipeline(
    modules={
        "input": input_component,
        "rewrite_template": rewrite_template,
        "llm": llm,
        "rewrite_retriever": retriever,
        "query_retriever": retriever,
        "join": argpack_component,
        "reranker": reranker,
        "response_component": response_component,
    },
    verbose=False,
)

# run both retrievers -- once with the hallucinated query, once with the real query
pipeline.add_link(
    "input", "rewrite_template", src_key="query_str", dest_key="query_str"
)
pipeline.add_link(
    "input",
    "rewrite_template",
    src_key="chat_history_str",
    dest_key="chat_history_str",
)
pipeline.add_link("rewrite_template", "llm")
pipeline.add_link("llm", "rewrite_retriever")
pipeline.add_link("input", "query_retriever", src_key="query_str")

# each input to the argpack component needs a dest key -- it can be anything
# then, the argpack component will pack all the inputs into a single list
pipeline.add_link("rewrite_retriever", "join", dest_key="rewrite_nodes")
pipeline.add_link("query_retriever", "join", dest_key="query_nodes")

# reranker needs the packed nodes and the query string
pipeline.add_link("join", "reranker", dest_key="nodes")
pipeline.add_link(
    "input", "reranker", src_key="query_str", dest_key="query_str"
)

# synthesizer needs the reranked nodes and query str
pipeline.add_link("reranker", "response_component", dest_key="nodes")
pipeline.add_link(
    "input", "response_component", src_key="query_str", dest_key="query_str"
)
pipeline.add_link(
    "input",
    "response_component",
    src_key="chat_history",
    dest_key="chat_history",
)

In [3]:
response = pipeline._execute("Quelles ont été les politiques de soutien à la pêche en Algerie?", pipeline_memory)

print(response)

Selon le Context Chunk 4, les politiques de soutien à la pêche en Algérie ont inclus :

* Le Plan National du Développement de la Pêche et de l'Aquaculture (PNDPA) 2003-2007
* Le Schéma Directeur de Développement des Activités de la Pêche et de l'Aquaculture (SDDAPA) 2025
* La feuille de route Strat-Saïd (2012-2014)
* Le plan Aqua-Pêche 2020

Ces politiques ont visé à promouvoir le développement de la pêche et de l'aquaculture en Algérie, en mettant en place des mécanismes de soutien pour les pêcheurs et les entreprises du secteur.


In [8]:
response = pipeline._execute("hello", pipeline_memory=pipeline_memory)

In [3]:
msg = ["hello", "how is food security in Algeria?", "What was my previous question?"]

for msg in msg:
    response = str(pipeline._run(msg, pipeline_memory=pipeline_memory))
    response.replace("assistant:", "").strip()
    print(response)



assistant: Bonjour! Comment puis-je vous aider aujourd'hui?
assistant: La sécurité alimentaire en Algérie est un sujet complexe et multiforme. Selon les contextes fournis, il semble que la sécurité alimentaire en Algérie est confrontée à plusieurs défis. 

D'une part, les politiques agricoles et de soutien à l'agriculture ont été mises en place pour améliorer la sécurité alimentaire, mais leur efficacité et leur impact sont discutés. Le Plan National de Développement Agricole (PNDA) de 2000 visait à améliorer la sécurité alimentaire, mais son évaluation est difficile en raison du manque de données et de documents formels.

D'autre part, les études et les recherches sur la sécurité alimentaire en Algérie sont nombreuses, mais elles montrent que le sujet est encore peu exploré, notamment en ce qui concerne les effets des politiques de soutien à l'agriculture et à la pêche sur la sécurité alimentaire.

En outre, les défis liés à la sécurité alimentaire en Algérie sont nombreux, tels que l

In [17]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(pipeline.dag)

In [18]:
# Save the network
net.write_html("../data/html/2_qpchat_dag.html")

In [3]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
